# Cloning repository and installing dependencies

In [ ]:
!git clone  https://github.com/pinellolab/DNA-Diffusion.git && cd DNA-Diffusion && uv sync

In [3]:
%cd DNA-Diffusion

/content/DNA-Diffusion


# Training

In [4]:
!uv run train.py -cn train_debug

model:
  _target_: src.dnadiffusion.models.unet.UNet
  dim: 200
  channels: 1
  dim_mults:
  - 1
  - 2
  - 4
  resnet_block_groups: 4
data:
  _target_: src.dnadiffusion.data.dataloader.get_dataset
  data_path: data/K562_hESCT0_HepG2_GM12878_12k_sequences_per_group.txt
  saved_data_path: data/encode_data.pkl
  load_saved_data: true
  debug: true
optimizer:
  _target_: torch.optim.Adam
  lr: 0.0001
diffusion:
  _target_: src.dnadiffusion.models.diffusion.Diffusion
  timesteps: 50
  beta_start: 0.0001
  beta_end: 0.2
training:
  distributed: false
  precision: float32
  num_workers: 1
  pin_memory: false
  batch_size: 1
  sample_batch_size: 1
  num_epochs: 2200
  min_epochs: 5
  patience: 2
  log_step: 50
  sample_epoch: 50000
  number_of_samples: 10
  use_wandb: false

  0% 0/2200 [00:00<?, ?it/s]/content/DNA-Diffusion/.venv/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In versio

# Running sampling with generated checkpoint

In [7]:
!uv run sample.py sampling.checkpoint_path='checkpoints/model_epoch5_step6_valloss_0.350965.pt' sampling.number_of_samples=1 sampling.sample_batch_size=1

model:
  _target_: src.dnadiffusion.models.unet.UNet
  dim: 200
  channels: 1
  dim_mults:
  - 1
  - 2
  - 4
  resnet_block_groups: 4
data:
  _target_: src.dnadiffusion.data.dataloader.get_dataset
  data_path: data/K562_hESCT0_HepG2_GM12878_12k_sequences_per_group.txt
  saved_data_path: data/encode_data.pkl
  load_saved_data: true
  debug: false
diffusion:
  _target_: src.dnadiffusion.models.diffusion.Diffusion
  timesteps: 50
  beta_start: 0.0001
  beta_end: 0.2
sampling:
  checkpoint_path: checkpoints/model_epoch5_step6_valloss_0.350965.pt
  sample_batch_size: 1
  number_of_samples: 1

(<src.dnadiffusion.data.dataloader.SequenceDataset object at 0x7b831aaabb30>, <src.dnadiffusion.data.dataloader.SequenceDataset object at 0x7b833d8ae900>, [1, 2, 3, 4], {1: 'GM12878_ENCLB441ZZZ', 2: 'HepG2_ENCLB029COU', 3: 'K562_ENCLB843GMH', 4: 'hESCT0_ENCLB449ZZZ'})
Loading checkpoint
Sending model to device
Generating 1 samples for cell GM12878_ENCLB441ZZZ
100% 1/1 [00:02<00:00,  2.38s/it]
Generatin

# Display generated sequences

In [13]:
import os
import subprocess

def display_sequences(output_dir="data/outputs"):
    if not os.path.isdir(output_dir):
        print(f"Error: Directory '{output_dir}' not found.")
        return

    print(f"Displaying sequences from: {output_dir}\n")

    for filename in sorted(os.listdir(output_dir)):
        filepath = os.path.join(output_dir, filename)

        if os.path.isfile(filepath) and "gitkeep" not in filepath:
          cell_type = filename.split('_')[0]
          print(f"--- Cell Type: {cell_type} ({filename}) ---")
          result = subprocess.run(['cat', filepath], capture_output=True, text=True, check=True)
          print(result.stdout)
          print("-" * (len(cell_type) + 18) + "\n")

display_sequences()

Displaying sequences from: data/outputs

--- Cell Type: GM12878 (GM12878_ENCLB441ZZZ.txt) ---
CGACCACTATCATCCTGCCCATACTTTCTCCGAGCTTGCGCTTAGTACTCGACCCGTCCTTAACAAACAGTCATCGTATTTGGTCGGGCCTAATACGGTTGGTGTTCTAAGCTTACAGTGATTACAGCGAGCTAGTTGACCATACGTATTTTGATTTGTTAATGCTACCCATGTTTATTCTACAAATCCATATCCGACAC
-------------------------

--- Cell Type: HepG2 (HepG2_ENCLB029COU.txt) ---
GGTACGCGTATCCGCGGAATCATTAACCAAAACCCGCTTTCATACAGTACTAAAACCATCGAACACACTAACCTGCGTACGAATATCATAAACACCAGATCCCGTGCACACGGTGCTCACCGCAACCAACCACCTAATGCGTAAACCATTCGGATACATAACAAGTGCACACACGCACGTAGGGGCCTAGTAATCAGCCC
-----------------------

--- Cell Type: K562 (K562_ENCLB843GMH.txt) ---
TAAAACAACTACCTAGAACATCTTAACACCCAACGTAACTTTCTTTCGTGTTTTTTTACACACACAGTAGAATAACTTCTTACAACGCAATCCCCGGCCAAGCCCTCATACATGTAGGTTGAACGAGTACGTATCAGCCAAGACGAAGTATCACAACATTATACACAGAACAGCCCGCGGGTAAACGTTATAAGTACCCA
----------------------

--- Cell Type: hESCT0 (hESCT0_ENCLB449ZZZ.txt) ---
GACCATCTACTTACTTTGTTTTCGTAACATATCAGTATTATCTTATTTTTTTCTTGAATTGTTTTTGTGCATACTTTCGT